In [242]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [243]:
data = pd.read_csv('Z:\\Downloads\\Gail_Share_Data\\GAIL.NS.csv')
data1 = pd.read_csv('Z:\\Downloads\\Gail_Share_Data\\gail_data.csv')

In [244]:
data = data.drop(['Adj Close'], axis=1)
data1 = data1.drop(['time'], axis=1)
data.columns = data.columns.str.lower()

In [245]:
data.head()

,date,open,high,low,close,volume
0,2023-05-05,107.349998,108.500000,107.050003,107.650002,4948142
1,2023-05-08,107.699997,108.099998,107.000000,107.449997,4868763
2,2023-05-09,107.449997,108.250000,106.550003,106.949997,11885359
3,2023-05-10,106.900002,109.099998,106.250000,109.000000,7172451
4,2023-05-11,109.849998,110.199997,108.500000,109.349998,7784358


In [246]:
Data = pd.concat([data, data1], axis=0)
Data.shape

(370791, 6)

In [247]:
data.columns

Index(['date', 'open', 'high', 'low', 'close', 'volume'], dtype='object')

In [248]:
data1.columns


Index(['date', 'open', 'high', 'low', 'close', 'volume'], dtype='object')

In [249]:
Data.isnull().sum()

date        0
open      141
high      141
low       141
close     141
volume    141
dtype: int64

In [250]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')

Data['open'] = imputer.fit_transform(Data[['open']])
Data['high'] = imputer.fit_transform(Data[['high']])
Data['low'] = imputer.fit_transform(Data[['low']])
Data['close'] = imputer.fit_transform(Data[['close']])
Data['volume'] = imputer.fit_transform(Data[['volume']])

In [251]:
Data.isnull().sum()

date      0
open      0
high      0
low       0
close     0
volume    0
dtype: int64

In [252]:
import DateTime as dt

Data['date'] = pd.to_datetime(Data['date'])
Data['year'] = Data['date'].dt.year
Data['month'] = Data['date'].dt.month
Data['day'] = Data['date'].dt.day
Data['day_of_week'] = Data['date'].dt.dayofweek
Data['day_of_year'] = Data['date'].dt.dayofyear
Data['quarter'] = Data['date'].dt.quarter
Data['day_name'] = Data['date'].dt.day_name()

In [253]:
Data.head()

,date,open,high,low,close,volume,year,month,day,day_of_week,day_of_year,quarter,day_name
0,2023-05-05,107.349998,108.500000,107.050003,107.650002,4948142.0,2023,5,5,4,125,2,Friday
1,2023-05-08,107.699997,108.099998,107.000000,107.449997,4868763.0,2023,5,8,0,128,2,Monday
2,2023-05-09,107.449997,108.250000,106.550003,106.949997,11885359.0,2023,5,9,1,129,2,Tuesday
3,2023-05-10,106.900002,109.099998,106.250000,109.000000,7172451.0,2023,5,10,2,130,2,Wednesday
4,2023-05-11,109.849998,110.199997,108.500000,109.349998,7784358.0,2023,5,11,3,131,2,Thursday


In [254]:
from sklearn.preprocessing import LabelEncoder
Data = Data.drop(['date'], axis=1)
Data['day_name'] = LabelEncoder().fit_transform(Data['day_name'])


In [255]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Data[['open', 'high', 'low', 'close', 'volume']] = scaler.fit_transform(Data[['open', 'high', 'low', 'close', 'volume']])
Data['volume'] = scaler.fit_transform(Data['volume'].values.reshape(-1, 1))

In [256]:
Data.head()

,open,high,low,close,volume,year,month,day,day_of_week,day_of_year,quarter,day_name
0,107.349998,108.500000,107.050003,107.650002,8.222298,2023,5,5,4,125,2,0
1,107.699997,108.099998,107.000000,107.449997,8.089457,2023,5,8,0,128,2,1
2,107.449997,108.250000,106.550003,106.949997,19.831804,2023,5,9,1,129,2,5
3,106.900002,109.099998,106.250000,109.000000,11.944703,2023,5,10,2,130,2,6
4,109.849998,110.199997,108.500000,109.349998,12.968736,2023,5,11,3,131,2,4


In [257]:
train = Data[Data['year'] < 2021]
test = Data[Data['year'] >= 2021]

X_train = train.drop(['close'], axis=1)
y_train = train['close']

X_test = test.drop(['close'], axis=1)
y_test = test['close']

In [258]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(370171, 11) (370171,)
(620, 11) (620,)


In [259]:


# for column in X_train.columns:
#     plt.figure(figsize=(10, 5))
#     sns.histplot(X_train[column], kde=True)
#     plt.title(f'Distribution of {column}')
#     plt.show()

In [260]:
from sklearn.preprocessing import PowerTransformer

# Applying Box-Cox Transform
pt = PowerTransformer()
x_train_transformed = pd.DataFrame(pt.fit_transform(X_train+0.000001), columns=X_train.columns)
x_train_transformed

,open,high,low,volume,year,month,day,day_of_week,day_of_year,quarter,day_name
0,-0.693277,-0.695510,-0.712560,0.601697,-1.341074,-1.708435,-1.728237,-1.511304,-2.063954,-1.384341,-0.926545
1,-0.713451,-0.708481,-0.711122,0.402589,-1.341074,-1.708435,-1.728237,-1.511304,-2.063954,-1.384341,-0.926545
2,-0.707698,-0.709920,-0.724047,0.291136,-1.341074,-1.708435,-1.728237,-1.511304,-2.063954,-1.384341,-0.926545
3,-0.722067,-0.715669,-0.719744,0.949923,-1.341074,-1.708435,-1.728237,-1.511304,-2.063954,-1.384341,-0.926545
4,-0.719197,-0.717106,-0.718308,-0.456129,-1.341074,-1.708435,-1.728237,-1.511304,-2.063954,-1.384341,-0.926545
...,...,...,...,...,...,...,...,...,...,...,...
370166,-0.719197,-0.717106,-0.718308,1.743096,1.333042,1.503332,1.589695,0.729960,1.566076,1.302918,0.381075
370167,-0.717761,-0.715669,-0.715435,1.703519,1.333042,1.503332,1.589695,0.729960,1.566076,1.302918,0.381075
370168,-0.717761,-0.715669,-0.715435,1.553580,1.333042,1.503332,1.589695,0.729960,1.566076,1.302918,0.381075
370169,-0.719197,-0.718541,-0.721179,1.553580,1.333042,1.503332,1.589695,0.729960,1.566076,1.302918,0.381075


In [261]:


# for column in x_train_transformed.columns:
#     plt.figure(figsize=(10, 5))
#     sns.histplot(x_train_transformed[column], kde=True)
#     plt.title(f'Distribution of {column}')
#     plt.show()

In [263]:
from xgboost import XGBRegressor
import joblib

model = XGBRegressor().fit(X_train, y_train)

joblib.dump(model,'gail_model.pkl')

ypred = model.predict(X_test)

In [264]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

MAE = mean_absolute_error(y_test, ypred)
MSE = mean_squared_error(y_test, ypred)
RMSE = np.sqrt(MSE)
R2 = r2_score(y_test, ypred)

print(f'Mean Absolute Error: {MAE}')
print(f'Mean Squared Error: {MSE}')
print(f'Root Mean Squared Error: {RMSE}')
print(f'R2 Score: {R2}')




Mean Absolute Error: 0.670437146785219
Mean Squared Error: 3.1598290897187074
Root Mean Squared Error: 1.7775908105406901
R2 Score: 0.9932724114328844


In [265]:
# First - without yeo-johnson transformer
# 
# Mean Absolute Error: 0.9273573091875628
# Mean Squared Error: 5.67327668454551
# Root Mean Squared Error: 2.3818641196645767
# R2 Score: 0.9888430862085151
# 
# 
# second - with yeo-johnson transformer
# 
# Mean Absolute Error: 64.5774778843498
# Mean Squared Error: 4579.253085197008
# Root Mean Squared Error: 67.6701786993134
# R2 Score: -8.005436318681314


# Changed Dependent Variable to Close
# 
# Mean Absolute Error: 0.670437146785219
# Mean Squared Error: 3.1598290897187074
# Root Mean Squared Error: 1.7775908105406901
# R2 Score: 0.9932724114328844


In [266]:
# from sklearn.model_selection import RandomizedSearchCV
# 
# params = {
#     'n_estimators': [100, 200, 300, 400, 500],
#     'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10],
#     'learning_rate': [0.05, 0.1, 0.15, 0.2],
#     'min_child_weight': [1, 2, 3, 4],
#     'colsample_bytree': [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
#     'gamma': [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
#     'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1]
# }
# a = RandomizedSearchCV(model, param_distributions=params, n_iter=5, n_jobs=-1, cv=5, verbose=3, random_state=42)

In [ ]:
# best = a.fit(X_train, y_train)
# best.best_params_